In [33]:
# !pip install langchain-google-genai==0.0.11
# !pip install langchain==0.1.12

In [8]:
import os
import yaml
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [3]:
with open('chatgpt_api_credentials.yml', 'r') as file:
    api_creds = yaml.safe_load(file)

In [4]:


os.environ['GOOGLE_API_KEY'] = api_creds['gemini_api_key']

## Load Google Gemini LLM

In [6]:
gemini_model = ChatGoogleGenerativeAI(model="gemini-pro",
                                      convert_system_message_to_human=True)

# create a basic chain

In [9]:
PROMPT = "tell me a joke about {topic}"
prompt = ChatPromptTemplate.from_template(PROMPT)
chain = (
         prompt
         |
         gemini_model
)

response = chain.invoke({"topic": "lion"})
print(response.content)

What do you call a lion that's always on the lookout for trouble?

A prowler!


In [10]:
# can be used on multiple prompts also
topics = [{'topic': 'AI'}, {'topic': 'Statistics'}]
responses = chain.map().invoke(topics)

In [12]:
for response in responses:
    print(response.content)
    print('-----')
#     print('\n')

Why did the AI cross the road?

To get to the other algorithm.
-----
Why did the statistician put his head in the oven?

To get the mean temperature.
-----


#### No-Conversation history?

In [14]:
prompt = ChatPromptTemplate.from_template("{query}")
basic_chain = (
               prompt
               |
              gemini_model
)

In [16]:
basic_chain.invoke({"query": "what are the first four colors of rainbow? "})


AIMessage(content='Red, Orange, Yellow, Green', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-d5afd021-f1a8-44db-9632-14f8907596a4-0')

In [17]:
response = basic_chain.invoke({"query" : 'What are the first four colors of the rainbow?'})
print(response.content)

1. Red
2. Orange
3. Yellow
4. Green


In [18]:
response = basic_chain.invoke({"query" : 'And the other three?'})
print(response.content) # gives a totally random response

The context does not mention the other three, so I cannot answer this question from the provided context.


# Add momory to build conversation chain

In [19]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [22]:
# create prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Act as a helpful AI Assistant"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)


In [21]:
# k=3 stores last 3 conversations between you and Gemini
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

ConversationBufferWindowMemory(return_messages=True, k=3)

In [23]:
memory.load_memory_variables({}) # shows the conversation history

{'history': []}

In [25]:
from operator import itemgetter

# creates the conversation chain
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables)
        |
        itemgetter("history")
    )
    |
    prompt
    |
    gemini_model
)

In [27]:
user_input = {'input': 'What are the first four colors of a rainbow'}
response = chain.invoke(user_input)
print(response.content)

1. Red
2. Orange
3. Yellow
4. Green


In [28]:
memory.load_memory_variables({})

{'history': []}

In [29]:
# remember to save your conversation to the memory
memory.save_context(user_input, {"output": response.content})
memory.load_memory_variables({}) # remembers the conversation history

{'history': [HumanMessage(content='What are the first four colors of a rainbow'),
  AIMessage(content='1. Red\n2. Orange\n3. Yellow\n4. Green')]}

In [31]:
user_input = {'input': 'And the last 3?'}
response = chain.invoke(user_input) # uses history of the past conversation to give a better response
print(response.content)

The last three colors of a rainbow are:

* Blue
* Indigo
* Violet


In [32]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='What are the first four colors of a rainbow'),
  AIMessage(content='1. Red\n2. Orange\n3. Yellow\n4. Green')]}